Connexion + création du data frame via la base de donnés Mysql

In [2]:
import pandas as pd 
import numpy as np 
import pyodbc

server = 'localhost' 
database = 'datananalysetest' 
username = 'root' 
password = ''

cnxn = pyodbc.connect(f"Driver={{MySQL ODBC 8.0 Unicode Driver}};Server={server};Database={database};Uid={username};Pwd={password};")
cursor = cnxn.cursor()

query = "SELECT * FROM airline_safety" 
df = pd.read_sql(query, cnxn)
print(df.head(26))



                    COL 1                   COL 2            COL 3  \
0                 airline  avail_seat_km_per_week  incidents_85_99   
1              Aer Lingus               320906734                2   
2               Aeroflot*              1197672318               76   
3   Aerolineas Argentinas               385803648                6   
4             Aeromexico*               596871813                3   
5              Air Canada              1865253802                2   
6              Air France              3004002661               14   
7              Air India*               869253552                2   
8        Air New Zealand*               710174817                3   
9        Alaska Airlines*               965346773                5   
10               Alitalia               698012498                7   
11     All Nippon Airways              1841234177                3   
12              American*              5228357340               21   
13      Austrian Air

c:\Users\isabe\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Utilisation maintenant de fichier excel , pour effectuer le traitement demander lors du stage .
Test avec 2 Metric Group ILOMN et Ressources State

ILOMN

In [8]:
import pandas as pd
import numpy as np

df_oem = pd.read_excel('C:/Users/isabe/Python/Excel/AlarmesOem.xlsx',engine='openpyxl')
dfapp_exa = pd.read_excel("C:/Users/isabe/Python/Excel/AppliExa.xlsx",engine='openpyxl')

df = df_oem[(df_oem['MetricGroup'] == 'ILOM SNMP Test Trap')]
for index,row in df.iterrows():
    host = df.at[index,'TargetName'].split('-',1)[0]
    code_app = (dfapp_exa.loc[(dfapp_exa['VM'].str.upper() == host.upper())]["CodeApplication"].head(1)).values
    if len(code_app) == 0 : 
        df.at[index,'CodeApplication'] = "XH_"
    else :
        df.at[index,'CodeApplication'] = code_app[0]
    df.at[index,'HOST'] = host
  
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.to_csv("./Sortie_CSV/Alarmes_OEM_2022_ILOMTest.csv")

C:\Users\isabe\AppData\Local\Temp\ipykernel_23440\1197120455.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index,'CodeApplication'] = "XH_"
C:\Users\isabe\AppData\Local\Temp\ipykernel_23440\1197120455.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index,'HOST'] = host


Metric Group Ressources State

In [2]:
import pandas as pd
import numpy as np

dfapp_exa = pd.read_excel("Excel/AppliExa.xlsx",engine='openpyxl')
df_oem = pd.read_excel('Excel/AlarmesOem.xlsx',engine='openpyxl')

df = df_oem[(df_oem ['MetricGroup'] == 'Resource State')] #Récupération du MetricGroup

for index, row in df.iterrows(): #Pour chaque index dans la colonne dans le dataframe
    df.at[index , 'Cluster'] = row['TargetName'] #On met les donnés de targeName à l'endroit spécifier
    cluster1 = row['TargetName']
    #On récupére le codeApplication ou le cluster est égale aux cluster de notre Alarmes et ensuite on récupére 
    #le CodeApplication . 
    code_app = (dfapp_exa.loc[(dfapp_exa['Cluster'].str.upper() == cluster1.upper()) ]["CodeApplication"]).head(1).values 

    if len(code_app) == 0 :
        code_app = (dfapp_exa.loc[(dfapp_exa['Cluster'].str.upper() == cluster1.upper()) ]["CodeApplication"]).head(1).values
        if len(code_app) == 0: #Donc si il y a plusieurs codeApplication on met XH_ 
            code_app = "XH_"   
        else : 
            code_app = str(code_app[0])
    else :
        code_app = code_app[0]
    df.at[index,'CodeApplication'] = code_app #On ecrit donc le code CodeApp à cet endroit

df.loc[df['TargetName'] == 'has_weya0333.net.intra.laposte.fr','CodeApplication'] = 'OM_'# Petite exeption dans lesquel j'attribue directement le codeApp
    
  
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] # Ceci permet juste de ne récupérer que notre metricGroup dans le fichier que l'on modifie 
df.to_csv("./Sortie_CSV/Alarmes_OEM_2022_ResourceState.csv")#,sheet_name="Sheet1")#On transforme le DataFrame en CSV

C:\Users\isabe\AppData\Local\Temp\ipykernel_13560\1851924740.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index , 'Cluster'] = row['TargetName'] #On met les donnés de targeName à l'endroit spécifier
C:\Users\isabe\AppData\Local\Temp\ipykernel_13560\1851924740.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index,'CodeApplication'] = code_app #On ecrit donc le code CodeApp à cet endroit


Script chapeau .

In [5]:
import subprocess
import pandas as pd



#Dossier où sont rangés tous les scripts individuels des metricsGroup
folder = './Script/'
#Importation fichier excel metricsGroup
df = pd.read_excel("Excel/Script.xlsx",engine='openpyxl')
#Pour chaque script dans la colonne Scripts




for i in df["Script"] :
    
    #Execute les scripts externe python
    subprocess.call(["python", folder+i])
    print("Script executed : "+folder+i)

    output = subprocess.check_output(["python", folder+i])
    print(output)

    
        
    
    #Vérifier l'output des scripts externes executés

Script executed : ./Script/RessourceState.py
b''
Script executed : ./Script/ILOMN.py
b''
